In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [7]:
import tensorflow.compat.v1 as tf

with tf.Session() as sess:
    x = tf.placeholder( tf.float32, [None, 784] )


In [8]:
#import tensorflow as tf

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras import optimizers
from pandas import Series
import math
import numpy
from tensorflow.keras.layers import *
from tensorflow.keras import layers
import pandas as pd

data = pd.read_csv("SeoulBikeData.csv",encoding='CP949')

In [25]:
import os
import random

# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(optimizer='adam'):
    first_input = Input(shape=(24,8))

    in1=Bidirectional(LSTM(130))(first_input)
    in1=Dense(1,activation='relu')(in1)
    model=Model(first_input,in1)
    model.compile(optimizer=optimizer, loss='mse',metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
#dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = trainX
Y = trainY
# create model
model = KerasClassifier(build_fn=create_model, epochs=3, batch_size=10, verbose=0)
# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))



Best: 0.146764 using {'optimizer': 'SGD'}
0.146764 (0.207556) with: {'optimizer': 'SGD'}
0.006300 (0.008910) with: {'optimizer': 'RMSprop'}
0.006300 (0.008910) with: {'optimizer': 'Adagrad'}
0.006300 (0.008910) with: {'optimizer': 'Adadelta'}
0.006300 (0.008910) with: {'optimizer': 'Adam'}
0.006300 (0.008910) with: {'optimizer': 'Adamax'}
0.006300 (0.008910) with: {'optimizer': 'Nadam'}


In [13]:
cols=data.columns[1:-3]
cols=cols.drop(['Visibility (10m)','Hour'])

In [14]:
from sklearn.preprocessing import MinMaxScaler
values = data[cols].values.reshape(-1,8)
values = values.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
#scaled = (values-(np.min(values)))/(np.max(values)-(np.min(values)))
scaled=scaler.fit_transform(values)

In [15]:
train_size = int(len(scaled) * 0.80)
val_size = int(len(scaled) * 0.10)

test_size = len(scaled) - train_size-val_size
train,val, test,Test = scaled[0:train_size,:],scaled[train_size:train_size+val_size,:], scaled[train_size+val_size:,:],scaled[train_size:,:]
print(len(train),len(val), len(test),len(Test))

7008 876 876 1752


In [16]:
def create_dataset_mul(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back, -4])
    print(len(dataY))
    return np.array(dataX), np.array(dataY)

In [17]:
look_back = 24
trainX, trainY = create_dataset_mul(train, look_back)
valX, valY = create_dataset_mul(val, look_back)
testX, testY = create_dataset_mul(test, look_back)

6984
852
852


In [15]:

### UTILITY FUNCTIONS FOR HYPERPARAM SEARCH ###

def set_seed_TF2(seed):
    
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)


def get_model(param):

    model = Sequential()
    model.add(LSTM(param['unit'], activation=param['act']))
    model.add(RepeatVector(target_seq))
    model.add(LSTM(param['unit'], activation=param['act'], return_sequences=True))
    model.add(TimeDistributed(Dense(1)))
    
    opt_choices = {'adam': Adam(),
                   'rms': RMSprop()}
    
    opt = opt_choices[param['opt']]
    opt.lr = param['lr'] 
    
    model.compile(opt, 'mse')
    
    return model

In [17]:
model = KerasClassifier(build_fn=get_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


NameError: name 'KerasClassifier' is not defined

In [4]:

### CREATE GRID FOR HYPERPARAM SEARCH ###

param_grid = {
    'unit': [128,64,32], 
    'lr': [1e-2,1e-3], 
    'act': ['elu','relu'], 
    'opt': ['adam','rms'],
    'epochs': 200,
    'batch_size': 512
}

In [ ]:
es = EarlyStopping(patience=10, verbose=0, min_delta=0.001, monitor='val_loss', mode='auto', restore_best_weights=True)

hypermodel = get_model

kgs = KerasGridSearch(hypermodel, param_grid, monitor='val_loss', greater_is_better=False, tuner_verbose=1)
kgs.set_seed(set_seed_TF2, seed=33)
kgs.search(X_train, y_train_raw, validation_data=(X_val, y_val_raw), callbacks=[es])